In [117]:
import geopandas as gpd
import pandas as pd

pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

# cities = gpd.read_feather(
#     "/Users/test/Documents/code/IL2/factory_loc_service/examples/closest_cities.feather"
# )
cities = gpd.read_parquet(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/cities.parquet"
)

# uinput_industry = "oil_and_gas_ext"
# uinput_spec_num = {"Машинист": 200, "Оператор, аппаратчик": 300}
city_names = cities["region_city"].values

In [118]:
# print(uinput_industry, uinput_spec_num, city_names)

In [119]:
ontology = pd.read_pickle("new_ontology.pkl")

grouped_grads = pd.read_pickle("grouped_grads.pkl")

cv = pd.read_parquet("../api/app/data/cv.gzip").rename(
    columns={"hh_name": "speciality"}
)
YEAR = 2021
cv = cv[cv["year"] == YEAR]

In [120]:
grouped_grads

graduates_per_year_forecast  id_cv  city_capacity_grads_and_cv_sum
cluster_center                 region_city                    type edu_group_code                                                                    
Адыгея, Адыгейск               Адыгея, Адыгейск               CV   5                                       NaN    2.0                             2.0
                                                                   11                                      NaN    3.0                             3.0
                                                                   15                                      NaN   21.0                            21.0
                                                                   18                                      NaN   12.0                            12.0
                                                                   19                                      NaN    2.0                             2.0
...                                                                                                        ...    ...                             ...
Ярославская область, Ярославль Ярославская область, Ярославль СПО  18                                     59.0    NaN                            59.0
                                                                   19                                    309.0    NaN                           309.0
                                                                   21                                    120.0    NaN                           120.0
                                                                   22                                     41.0    NaN                            41.0
                                                                   33                                     10.0    NaN                            10.0

[68338 rows x 3 columns]

In [121]:
mask = grouped_grads.reset_index(drop=False).loc[:, "cluster_center"].isin(city_names)
grouped_grads = (
    grouped_grads.reset_index(drop=False)
    .loc[mask, :]
    .set_index(["cluster_center", "region_city", "type", "edu_group_code"])
)

# mask = cv["region_city"].isin(city_names)
# cv = cv[mask]

In [122]:
# ontology = pd.read_csv(
#     "/Users/test/Documents/code/IL2/industrial-location/api/app/data/ontology.csv",
#     index_col=0,
# )

# ontology.loc[ontology["industry_code"] == "mining_oil_gas", "industry_code"] = (
#     "oil_and_gas_ext"
# )
# ontology.loc[ontology["industry_code"] == "mmining_coal_ores", "industry_code"] = (
#     "coal_and_metal_ext"
# )
# ontology.loc[ontology["industry_code"] == "mech_engineering", "industry_code"] = (
#     "machinery"
# )
# ontology.loc[ontology["industry_code"] == "shipbuilding", "industry_code"] = (
#     "shipbuilding"
# )
# ontology.loc[ontology["industry_code"] == "aircraft_engineering", "industry_code"] = (
#     "aircraft_and_space"
# )
# ontology.loc[ontology["industry_code"] == "non_ferrous_metallurgy", "industry_code"] = (
#     "nonferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "ferrous_metallurgy", "industry_code"] = (
#     "ferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "chemical", "industry_code"] = "chemicals"
# ontology.loc[ontology["industry_code"] == "pharma", "industry_code"] = "pharmacy"
# ontology.loc[ontology["industry_code"] == "electronics", "industry_code"] = (
#     "electronics"
# )

In [123]:
# ontology["industry_code"].unique()

In [124]:
# ontology.to_pickle("new_ontology.pkl")

In [ ]:
cv = cv.merge(ontology[["speciality", "edu_group_code"]], on="speciality")
cv["type"] = "CV"
grouped_cv = (
    cv.groupby(
        ["cluster_center", "region_city", "type", "edu_group_code", "speciality"]
    )["id_cv"]
    .count()
    .to_frame()
).rename(columns={"id_cv": "cv_count"})
cv.head(3)

,id_cv,speciality,year,region_city,cluster_center,edu_group_code,type
0,c489ca40-355c-11eb-ac41-3bfa22f2d66b,Сварщик,2021,"Башкортостан, Туймазы","Башкортостан, Туймазы",15,CV
1,c489ca40-355c-11eb-ac41-3bfa22f2d66b,Сварщик,2021,"Башкортостан, Туймазы","Башкортостан, Туймазы",15,CV
2,c489ca40-355c-11eb-ac41-3bfa22f2d66b,Сварщик,2021,"Башкортостан, Туймазы","Башкортостан, Туймазы",15,CV


In [126]:
grouped_cv

cv_count
cluster_center                 region_city                    type edu_group_code speciality                     
Адыгея, Адыгейск               Адыгея, Адыгейск               CV   5              Геолог                        2
                                                                   11             Оператор, аппаратчик          2
                                                                                  Электромонтажник              1
                                                                   15             Инженер-проектировщик         1
                                                                                  Оператор, аппаратчик          4
...                                                                                                           ...
Ярославская область, Ярославль Ярославская область, Ярославль CV   26             Технолог                      3
                                                                   27             Инженер по качеству          48
                                                                   33             Исследователь                19
                                                                                  Оператор, аппаратчик         37
                                                                                  Технолог                      3

[106678 rows x 1 columns]

In [ ]:
ontology["speciality"].unique()

array(['Оператор, аппаратчик', 'Монтажник', 'Машинист',
       'Мастер по ремонту оборудования', 'Разнорабочий', 'Слесарь',
       'Геодезист', 'Геолог', 'Технолог', 'Инженер-эколог',
       'Инженер-конструктор', 'Сварщик', 'Наладчик',
       'Токарь, фрезеровщик, шлифовщик', 'Контролер ОТК',
       'Инженер-проектировщик', 'Специалист по сертификации',
       'Инженер по качеству', 'Механик', 'Инженер по эксплуатации',
       'Лаборант', 'Исследователь', 'Электромонтажник'], dtype=object)

In [ ]:
# grads = pd.read_csv(
#     "/Users/test/Documents/code/IL2/factory_loc_service/examples/graduates.csv"
# )

# grads.dropna(subset="edu_group", inplace=True)
# grouped_grads = (
#     grads.groupby(["cluster_center", "region_city", "type", "edu_group_code"])[
#         "graduates_per_year_forecast"
#     ]
#     .sum()
#     .to_frame()
# )
# grouped_grads = grouped_grads.join(grouped_cv, how="outer")
# grouped_grads["city_capacity_grads_and_cv_sum"] = grouped_grads[
#     "graduates_per_year_forecast"
# ].fillna(0) + grouped_grads["cv_count"].fillna(0)
# grouped_grads.to_pickle("grouped_grads.pkl")

In [128]:
grouped_grads

graduates_per_year_forecast  cv_count  city_capacity_grads_and_cv_sum
cluster_center                 region_city                    type edu_group_code speciality                                                                                  
Адыгея, Адыгейск               Адыгея, Адыгейск               CV   5              Геолог                                         NaN       2.0                             2.0
                                                                   11             Оператор, аппаратчик                           NaN       2.0                             2.0
                                                                                  Электромонтажник                               NaN       1.0                             1.0
                                                                   15             Инженер-проектировщик                          NaN       1.0                             1.0
                                                                                  Оператор, аппаратчик                           NaN       4.0                             4.0
...                                                                                                                              ...       ...                             ...
Ярославская область, Ярославль Ярославская область, Ярославль СПО  18             NaN                                           59.0       NaN                            59.0
                                                                   19             NaN                                          309.0       NaN                           309.0
                                                                   21             NaN                                          120.0       NaN                           120.0
                                                                   22             NaN                                           41.0       NaN                            41.0
                                                                   33             NaN                                           10.0       NaN                            10.0

[110621 rows x 3 columns]

In [20]:
# --- START ---
"""
Эта штука завязана на входных профессиях и индустрии,
поэтому нужно каждый раз пересчитывать -- мб можно оптимизировать и пересчитывать только часть
но имхо оно и так быстро
"""

uinput_spec_num_2 = dict()
competitor_industries = list()
competitor_fatories = list()


for k in uinput_spec_num.keys():
    uinput_spec_num_2[k] = (
        ontology.loc[ontology["speciality"] == k, "edu_group_id"]
        .drop_duplicates()
        .values.tolist()
    )
    competitor_industries += (
        ontology.loc[ontology["speciality"] == k, "industry_code"]
        .drop_duplicates()
        .tolist()
    )
# --- START ---

competitor_industries = set(competitor_industries)
competitor_industries.remove(uinput_industry)


grad_col = []
fact_col = []

for col in cities.columns:
    if "factor" in col and uinput_industry in col:
        fact_col.append(col)

    if "factor" in col and any(industry in col for industry in competitor_industries):
        competitor_fatories.append(col)

    if "grad" in col and uinput_industry in col:
        grad_col.append(col)

cities["competitors_factories_num"] = cities[competitor_fatories].sum(axis=1)

print(competitor_industries)
print(grad_col, fact_col, competitor_fatories)

{'mining_coal_ores', 'ferrous_metallurgy', 'pharmacy', 'machinery', 'shipbuilding', 'nonferrous_metallurgy', 'chemicals', 'electronics'}
['graduates_oil_and_gas_ext'] ['factories_oil_and_gas_ext'] ['factories_machinery', 'factories_shipbuilding', 'factories_pharmacy', 'factories_chemicals', 'factories_nonferrous_metallurgy', 'factories_ferrous_metallurgy', 'factories_electronics']


In [97]:
grouped_grads

graduates_per_year_forecast  id_cv  city_capacity_grads_and_cv_sum
cluster_center                 region_city                    type edu_group_code                                                                    
Адыгея, Адыгейск               Адыгея, Адыгейск               CV   5                                       NaN    2.0                             2.0
                                                                   11                                      NaN    3.0                             3.0
                                                                   15                                      NaN   21.0                            21.0
                                                                   18                                      NaN   12.0                            12.0
                                                                   19                                      NaN    2.0                             2.0
...                                                                                                        ...    ...                             ...
Ярославская область, Ярославль Ярославская область, Ярославль СПО  18                                     59.0    NaN                            59.0
                                                                   19                                    309.0    NaN                           309.0
                                                                   21                                    120.0    NaN                           120.0
                                                                   22                                     41.0    NaN                            41.0
                                                                   33                                     10.0    NaN                            10.0

[68217 rows x 3 columns]

In [ ]:
grouped_grads = (
    grouped_grads.reset_index(drop=False)
    .groupby(["cluster_center", "type"])[["city_capacity_grads_and_cv_sum", "count"]]
    .sum()
    .join(
        cities[
            ["region_city", "population", "num_in_migration"]
            + grad_col
            + fact_col
            + ["factories_total"]
            + ["competitors_factories_num"]
        ]
        .rename(columns={"region_city": "cluster_center"})
        .set_index("cluster_center"),
        how="left",
    )
)

grouped_grads[
    [
        "num_in_migration",
        "graduates_oil_and_gas_ext",
        "factories_oil_and_gas_ext",
        "factories_total",
        "city_capacity_grads_and_cv_sum",
        "population",
        "competitors_factories_num",
    ]
] = (
    grouped_grads[
        [
            "num_in_migration",
            "graduates_oil_and_gas_ext",
            "factories_oil_and_gas_ext",
            "factories_total",
            "city_capacity_grads_and_cv_sum",
            "population",
            "competitors_factories_num",
        ]
    ]
    .round(0)
    .fillna(0)
    .astype(int)
)

grouped_grads["working_population"] = (
    (grouped_grads["population"] * 0.65).round(0).astype(int)
)

In [30]:
"""
(!) Сейчас есть всё кроме непосредственной оценки (!)
А ну и задаваемое кол-во специалистов пока не учитывается
Но это следствие из отсутствия реализации расчета оценки

Эту таблицу практически в таком виде и планирую отдавать

Из того что обсуждали с СА:
условно выпускников взвесить по кол-ву предприятий тк их число ту мач большое
"""

'\n(!) Сейчас есть всё кроме непосредственной оценки (!)\nА ну и задаваемое кол-во специалистов пока не учитывается\nНо это следствие из отсутствия реализации расчета оценки\n\nЭту таблицу практически в таком виде и планирую отдавать\n\nИз того что обсуждали с СА:\nусловно выпускников взвесить по кол-ву предприятий тк их число ту мач большое\n'

In [15]:
grouped_grads

city_capacity_grads_and_cv_sum   id_cv  population  num_in_migration  graduates_oil_and_gas_ext  factories_oil_and_gas_ext  factories_total  competitors_factories_num  working_population
cluster_center                 type                                                                                                                                                                                            
Адыгея, Адыгейск               CV                               118   118.0       13191                 4                          0                          0                0                          0                8574
Адыгея, Майкоп                 CV                               260   260.0      139687                20                        554                          0                6                          5               90797
                               ВПО                             1993     0.0      139687                20                        554                          0                6                          5               90797
                               СПО                              660     0.0      139687                20                        554                          0                6                          5               90797
Алтай, Горно-Алтайск           CV                               508   508.0       64957                 7                        468                          0                1                          1               42222
...                                                             ...     ...         ...               ...                        ...                        ...              ...                        ...                 ...
Ярославская область, Углич     CV                               226   226.0       32228                 4                          0                          0                5                          5               20948
                               СПО                               40     0.0       32228                 4                          0                          0                5                          5               20948
Ярославская область, Ярославль CV                              2134  2134.0      570824               443                        559                          0               65                         65              371036
                               ВПО                             5723     0.0      570824               443                        559                          0               65                         65              371036
                               СПО                             1307     0.0      570824               443                        559                          0               65                         65              371036

[2111 rows x 9 columns]

In [32]:
""" ------------------------- FINISH HERE -------------------------------- """

' ------------------------- FINISH HERE -------------------------------- '

In [ ]:
gg = pd.read_pickle(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/grouped_grads.pkl"
)

In [4]:
gg

,cluster_center,type,total,population,num_in_migration,edu_total
0,"Адыгея, Адыгейск",CV,118.0,13191.0,3.583744,0.0
1,"Адыгея, Майкоп",CV,260.0,139687.0,20.385065,2653.0
2,"Адыгея, Майкоп",ВПО,1993.0,139687.0,20.385065,2653.0
3,"Адыгея, Майкоп",СПО,660.0,139687.0,20.385065,2653.0
4,"Алтай, Горно-Алтайск",CV,508.0,64957.0,7.024049,1297.0
...,...,...,...,...,...,...
2109,"Ярославская область, Углич",CV,226.0,32228.0,3.897473,40.0
2110,"Ярославская область, Углич",СПО,40.0,32228.0,3.897473,40.0
2111,"Ярославская область, Ярославль",CV,2134.0,570824.0,442.679901,7030.0
2112,"Ярославская область, Ярославль",ВПО,5723.0,570824.0,442.679901,7030.0
